In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('phish_score.csv',header=None)
df=df.reset_index(drop=True)


In [3]:
df = df.sample(n = 5000, random_state = 12).copy()

In [4]:
df.drop(df.columns[[0,1,3]],axis=1,inplace=True)

In [5]:
df.columns=["Urls"]
df.head()

,Urls
35019,https://microeaggle.github.io/mail-update-vali...
4930,https://shaw-103506.weeblysite.com/
10932,https://qeuvev-f3cbb3.ingress-haven.ewp.live/w...
15502,https://us17.list-manage.com/survey?u=dc89f811...
10990,https://pub-162fe054277a40a08dd9cc160141da3c.r...


In [6]:
data=pd.read_csv('valid.csv',header=None)
data = data.sample(n = 5000, random_state = 12).copy()

In [7]:
data.columns=['Urls']

In [8]:
legiurl=data.sample(n=5000,random_state=12).copy()

In [9]:
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,Urls
0,http://babal.net/games/view/723/%D8%A7%D9%84%D...
1,http://maybank2u.com.my/mbb_info/m2u/public/pe...
2,http://buzzfil.net/article/5674/c-mignon/la-pe...
3,http://udn.com/news/story/6999/899717-3A%E9%80...
4,http://metro.co.uk/2015/04/17/harry-styles-jum...


In [10]:
 #ADDRESS BAR BASED FEATURES
   
 def getDomain(url):  
    domain = urlparse(url).netloc
    if re.match(r"^www.",domain):
            domain = domain.replace("www.","")
    return domain
    
    #Checks the presence of @(phishing) in URL (Have_At)
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = 0    
  return at

In [11]:
def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

In [12]:
#ip IP add. in url then phising
import ipaddress
def havingIP(url):
    try:
        ipaddress.ip_address(url)
        ip=1
    except:
        ip=0
    return ip    
    


In [13]:
#calculate number of subpages

from urllib.parse import urlparse,urlencode
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [14]:
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
    pos=url.rfind('//')
    if pos>6:
        if pos>7:
            return 1
        else:
            return 0
    else:
        return 0

In [15]:
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [16]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [17]:
# 8. Checking for Shortening Services in URL (Tiny_URL)
import re
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [18]:
#- in domain name is used in phising
def prefixsuffix(url):
    if'-' in urlparse(url).netloc:
        return 1
    else:
        return 0

In [19]:
!pip install python-whois

Defaulting to user installation because normal site-packages is not writeable


In [20]:
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime

In [21]:
def web_traffic(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 0

In [22]:
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
      age = 0
  return age

In [23]:
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end

In [24]:
#HTML AND JAVASCRIPT BASED FEATURES
# 15. IFrame Redirection (iFrame)
def iframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[<iframe>|<frameBorder>]", response.text):
          return 0
      else:
          return 1

In [25]:
# 16.Checks the effect of mouse over on status bar (Mouse_Over)
def mouseOver(response): 
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1
    else:
      return 0

In [26]:
# 17.Checks the status of the right click attribute (Right_Click)
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

In [27]:
# 18.Checks the number of forwardings (Web_Forwards)    
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

In [28]:
def featureExtraction(url,label):

    features = []
  #Address bar based features (10)
    features.append(getDomain(url))
    features.append(havingIP(url))
    features.append(haveAtSign(url))
    features.append(getLength(url))
    features.append(getDepth(url))
    features.append(redirection(url))
    features.append(httpDomain(url))
    features.append(tinyURL(url))
    features.append(prefixsuffix(url))
    try:
        response = requests.get(url)
    except:
        response = ""
        features.append(iframe(response))
        features.append(mouseOver(response))
        features.append(rightClick(response))
        features.append(forwarding(response))
        features.append(label)
  
    return features

In [29]:
legi_features = []
label = 0

for i in range(0, 5000):
  url = legiurl['Urls'].iloc[i]  
  legi_features.append(featureExtraction(url,label))

In [30]:
#converting the list to dataframe
feature_names = [ 'Domain_Name','Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix',  
                       'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()

,Domain_Name,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,babal.net,0,0,1,4,0,0,0,0,1,1,1,1,0
1,maybank2u.com.my,0,0,1,4,0,0,0,0,1,1,1,1,0
2,buzzfil.net,0,0,1,4,0,0,0,0,1,1,1,1,0
3,udn.com,0,0,1,4,0,0,0,0,1,1,1,1,0
4,metro.co.uk,0,0,1,4,0,0,0,0,1,1,1,1,0


In [31]:
df.shape

(5000, 1)

In [32]:
phish_features = []
label = 1
for i in range(0, 5000):
  url = df['Urls'].iloc[i]
  phish_features.append(featureExtraction(url,label))

In [33]:
feature_names = ['Domain_Name', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix',  
                       'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,Domain_Name,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,microeaggle.github.io,0,0,0,1,0,0,0,0,1,1,1,1,1
1,shaw-103506.weeblysite.com,0,0,0,0,0,0,0,1,1,1,1,1,1
2,qeuvev-f3cbb3.ingress-haven.ewp.live,0,0,1,6,0,0,0,1,1,1,1,1,1
3,us17.list-manage.com,0,0,1,1,0,0,0,1,1,1,1,1,1
4,pub-162fe054277a40a08dd9cc160141da3c.r2.dev,0,0,1,1,0,0,0,1,1,1,1,1,1


In [34]:
phishing.to_csv('phishing.csv', index= False)
legitimate.to_csv('legitimate.csv', index= False)

In [35]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.head()

,Domain_Name,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,babal.net,0,0,1,4,0,0,0,0,1,1,1,1,0
1,maybank2u.com.my,0,0,1,4,0,0,0,0,1,1,1,1,0
2,buzzfil.net,0,0,1,4,0,0,0,0,1,1,1,1,0
3,udn.com,0,0,1,4,0,0,0,0,1,1,1,1,0
4,metro.co.uk,0,0,1,4,0,0,0,0,1,1,1,1,0


In [36]:
urldata.shape

(10000, 14)

In [37]:
urldata.tail()

,Domain_Name,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
9995,page-support-appeal-review.d26vmbpac1b100.ampl...,0,0,1,1,0,0,0,1,1,1,1,1,1
9996,makemillion.website.ytwwrntym.com,0,0,0,0,0,0,0,0,1,1,1,1,1
9997,home-102925-102184.weeblysite.com,0,0,0,0,0,0,0,1,1,1,1,1,1
9998,attnet-102588.weeblysite.com,0,0,0,0,0,0,0,1,1,1,1,1,1
9999,mail.mibcopplr.site,0,0,0,2,0,0,0,0,1,1,1,1,1


In [38]:
urldata.to_csv('urldata.csv', index=False)

In [39]:
from IPython.display import FileLink

# Assuming 'urldata.csv' is in the current directory
FileLink(r'urldata.csv')

C:\Users\HP\OneDrive\Documents\jupyter\urldata.csv